In [54]:
#define method to create folders
def make_folder(foldername):
    if not os.path.exists(foldername):
        # create the directory if it doesn't exist
        os.makedirs(foldername)
    return foldername

In [66]:
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.transform import resize, rotate

# Path to the folder containing the MRI files
input_path = r'D:\DATASET\Dimensia\Input'
output_path = r'D:\DATASET\Dimensia\Output'

# check if the directory exists
make_folder(output_path)

for root, dirs, files in tqdm(os.walk(input_path), desc="Processing"):
        for file in files:
            if file.endswith('.img') and "nifti" in file:
                output_filename= os.path.splitext(file)[0]
                
                # Load the header and image data
                image_path = os.path.join(root, file)

                img = nib.load(image_path)
                data = img.get_fdata()

                # Split the path and get the last two components
                folder_name = os.path.basename(root.split(os.path.sep)[-3])
                output_folder_name = make_folder(os.path.join(output_path, folder_name))
                
                # Iterate over each slice and save as PNG
                for i in range(data.shape[2]):
                    if i == 67:
                        slice_data = data[:, :, i]  # Extract the slice data

                        # Rotate the image by -90 degrees
                        rotated_slice = rotate(slice_data, 90, resize=False)

                         # Pad the image to achieve desired size
                        pad_height = (256 - rotated_slice.shape[0]) // 2
                        pad_width = (256 - rotated_slice.shape[1]) // 2
                        pad_widths = ((pad_height, pad_height), (pad_width, pad_width), (0, 0))  # Add a new dimension for padding
                        padded_slice = np.pad(rotated_slice, pad_widths, mode='constant')

                        # Resize the image to 256x256
                        resized_slice = resize(padded_slice, (256, 256), preserve_range=True).astype(np.float32)

                        plt.imshow(resized_slice, cmap='gray')  # Display the slice
                        plt.axis('off')  # Disable axis
                        plt.savefig(os.path.join(output_folder_name, f'{output_filename}.png'), bbox_inches='tight', pad_inches=0, metadata={'Software': 'Matplotlib'})  # Save as PNG
                        plt.close()

Processing: 3101it [02:45, 18.75it/s]
